In [1]:
import os, json
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(os.environ['SQLALCHEMY_DATABASE_URI'])

data = {x: pd.read_sql_table(x, engine) for x in ['tool', 'parameter', 'parameter_value']}


In [2]:
data['tool'].head()

,id,tool_string,tool_name,tool_description,introduction,methods,reference,reference_link,default_selected,section_fk,input,display,result_description,video_tutorial,requires_signature
0,1,pca,PCA,Linear dimensionality reduction technique to v...,Principal Component Analysis (PCA) is a statis...,Principal Component Analysis was performed usi...,"Pearson, K. (1901). <b>LIII. On lines and plan...",https://doi.org/10.1080/14786440109462720,1,1,dataset,1,"The PCA plug-in embeds an interactive, three-d...",https://www.youtube.com/watch?v=r7WYtxeAaDw,0
1,2,tsne,t-SNE,Nonlinear dimensionality reduction technique t...,t-distributed Stochastic Neighbor Embedding (t...,t-SNE analysis was performed using the `TSNE` ...,L.J.P. van der Maaten and G.E. Hinton. <b>Visu...,https://lvdmaaten.github.io/publications/paper...,0,1,dataset,0,,None,0
2,3,clustergrammer,Clustergrammer,Interactive hierarchical clustering heatmap vi...,Clustergrammer is a web-based tool for visuali...,The interactive heatmap was generated using Cl...,"Fernandez, N.F., Gundersen, G.W., Rahman, A., ...",http://dx.doi.org/10.1038/sdata.2017.151,1,1,dataset,1,The Clustergrammer plug-in embeds an interacti...,https://youtu.be/8MgvyeroW98?t=59s,0
3,4,library_size_analysis,Library Size Analysis,Analysis of readcount distribution for the sam...,In order to quantify gene expression in an RNA...,Read counts were calculated by performing the ...,,,1,1,dataset,1,The Library Size Analysis plug-in embeds an in...,None,0
4,5,signature_table,Differential Expression Table,Differential expression analysis between two g...,Gene expression signatures are alterations in ...,The gene expression signature was generated by...,,,0,2,signature,1,The Signature Table plug-in embeds an interact...,None,1


In [3]:
data['parameter'].head()

,id,parameter_name,parameter_description,parameter_string,tool_fk
0,2,Top Genes,Number of most variably expressed genes to use...,nr_genes,1
1,3,Normalization,Normalization method for the dataset,normalization,1
2,5,Top Genes,Number of most variably expressed genes to use...,nr_genes,2
3,6,Normalization,Normalization method for the dataset,normalization,2
4,7,Top Genes,Number of most variably expressed genes to use...,nr_genes,3


In [4]:
data['parameter_value'].head()

,id,parameter_fk,value,type,default
0,3,2,500,int,0
1,4,2,2500,int,1
2,5,2,5000,int,0
3,6,3,logCPM,str,1
4,7,3,quantile,str,0
